In [1]:
#Загрузка библиотек
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
#Загрузка датасета
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
#Разделение датасета на выборки
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [11]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [12]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

*Логистическая регрессия*

In [13]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [17]:
#Обучение модели и вывод точности
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

print('Accuracy: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy: 0.72


In [16]:
#Применим крос-валидацию и переобучим модель с выводом точности
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

print('Accuracy: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy: 0.72


In [19]:
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838


In [24]:
from sklearn.metrics import roc_auc_score, log_loss

auc = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
loss = log_loss(y_true=y_test, y_pred=classifier.predict_proba(X_test)[:,1])
print(auc)
print(loss)

0.7840347790421852
0.5779604008230663


In [25]:
itog = pd.DataFrame({'metod': ['logReg'], 'Threshold': [thresholds[ix]],
                     'F-Score': [fscore[ix]],
                    'Precision': [precision[ix]],
                    'Recall': [recall[ix]],
                    'auc': [auc],
                    'loss': [loss]})
itog

,metod,Threshold,F-Score,Precision,Recall,auc,loss
0,logReg,0.386937,0.730323,0.647431,0.837558,0.784035,0.57796


*XGBoost*

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
#from xgboost import XGBClassifie

In [32]:
classifier_gb = Pipeline([
    ('features',feats),
    ('classifier_gb', GradientBoostingClassifier(random_state = 42)),
])


In [37]:
#Обучение модели
classifier_gb.fit(X_train, y_train)
y_score = classifier_gb.predict_proba(X_test)[:, 1]


In [38]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.394947, F-Score=0.740, Precision=0.698, Recall=0.788


In [40]:
auc = roc_auc_score(y_true=y_test, y_score=classifier_gb.predict_proba(X_test)[:,1])
loss = log_loss(y_true=y_test, y_pred=classifier_gb.predict_proba(X_test)[:,1])
print(auc)
print(loss)

0.8026153641179974
0.5397460438742135


In [52]:
itog = itog.append({'metod': 'XGBoost', 'Threshold': thresholds[ix],
                     'F-Score': fscore[ix],
                    'Precision': precision[ix],
                    'Recall': recall[ix],
                    'auc': auc,
                    'loss': loss}, ignore_index=True)


itog

,metod,Threshold,F-Score,Precision,Recall,auc,loss
0,logReg,0.386937,0.730323,0.647431,0.837558,0.784035,0.57796
1,XGBoost,0.394947,0.740248,0.697848,0.788134,0.802615,0.539746


*Дерево решений*

In [69]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из сотни деревьев
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')


In [70]:
classifier_dr2 = Pipeline([
    ('features',feats),
    ('classifier_dr2', model),
])

In [71]:
#Обучение модели
classifier_dr2.fit(X_train, y_train)
y_score = classifier_dr2.predict_proba(X_test)[:, 1]
print('Accuracy: {:.2f}'.format(classifier_dr2.score(X_test, y_test)))

Accuracy: 0.71


In [72]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.380000, F-Score=0.720, Precision=0.660, Recall=0.792


In [73]:
auc = roc_auc_score(y_true=y_test, y_score=classifier_dr2.predict_proba(X_test)[:,1])
loss = log_loss(y_true=y_test, y_pred=classifier_dr2.predict_proba(X_test)[:,1])
print(auc)
print(loss)

0.7714545257427087
0.6008009860061297


In [75]:
itog = itog.append({'metod': 'Tree', 'Threshold': thresholds[ix],
                     'F-Score': fscore[ix],
                    'Precision': precision[ix],
                    'Recall': recall[ix],
                    'auc': auc,
                    'loss': loss}, ignore_index=True)


itog

,metod,Threshold,F-Score,Precision,Recall,auc,loss
0,logReg,0.386937,0.730323,0.647431,0.837558,0.784035,0.57796
1,XGBoost,0.394947,0.740248,0.697848,0.788134,0.802615,0.539746
2,Tree,0.38,0.720021,0.65976,0.792396,0.771455,0.600801


Все модели показали примерно одинаковые показатели.


В метрике roc_auc_curve при сильном дисбалансе изменение кривой несущественно, соответственно данные будут показаны излишне оптимистическими. При дисбалансе классов приумещественно использовать precision_recall_curve метрику, так формула расчета использует точность, а не FPR.